In [1]:
# ! pip install --upgrade ultralytics

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import os
from collections import Counter
import Levenshtein

In [3]:
# Load a model
model = YOLO("yolov8m.pt")

In [4]:
# # Use the model
# model.train(data="charDetection-3/data.yaml", epochs=17)  # train the model
# metrics = model.val()  # evaluate model performance on the validation set

In [5]:
import cv2
import numpy as np
import os

input_folder = "Satriadata/Test/"
output_folder = "Satriadata/preprocessing/"
os.makedirs(output_folder, exist_ok=True)

target_size = 640

# Fungsi: resize letterbox agar seragam 640x640
def letterbox_resize(image, target_size=640):
    h, w = image.shape[:2]
    scale = target_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    canvas = np.full((target_size, target_size, 3), 128, dtype=np.uint8)  # abu-abu background
    top = (target_size - new_h) // 2
    left = (target_size - new_w) // 2
    canvas[top:top+new_h, left:left+new_w] = resized
    return canvas

# Fungsi: tingkatkan brightness dengan gamma ringan
def adjust_gamma(image, gamma=1.1):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

# Fungsi: naikkan kontras dengan CLAHE ringan
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# Fungsi: sharpening lembut
def sharpen(image):
    blur = cv2.GaussianBlur(image, (0,0), 1.0)
    return cv2.addWeighted(image, 1.2, blur, -0.2, 0)

# Pipeline utama
def preprocess_soft(img):
    img = letterbox_resize(img, target_size)  # seragamkan ukuran
    img = adjust_gamma(img, gamma=1.1)        # agak terang
    img = enhance_contrast(img)               # kontras naik sedikit
    img = sharpen(img)                        # teks lebih menonjol
    return img

# Proses semua gambar
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png')):
        path = os.path.join(input_folder, filename)
        img = cv2.imread(path)
        if img is None:
            print(f"Gagal membaca {filename}, dilewati.")
            continue

        processed = preprocess_soft(img)
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, processed)
        print(f"Hasil preprocessing disimpan: {save_path}")


Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest1.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest10.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest100.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest11.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest12.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest13.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest14.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest15.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest16.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest17.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest18.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest19.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest2.png
Hasil preprocessing disimpan: Satriadata/preprocessing/DataTest20.png
Hasil preprocessing d

In [6]:

# load model hasil training
model = YOLO("runs/detect/train7/weights/best.pt") 
# model = YOLO("charDetection-2\ep30.pt")  # ganti sesuai path model

# baca data dari CSV
df = pd.read_csv("Satriadata/Test/DataTest1.csv", sep=';')   # pastikan kolomnya "Name of File" dan "kunci"
# cari kolom otomatis (case insensitive)
col_name = [c for c in df.columns if "name" in c.lower()][1]
col_kunci = [c for c in df.columns if "kunci" in c.lower()][0]

print("Kolom nama file:", col_name)
print("Kolom kunci    :", col_kunci)

results_list = []

for _, row in df.iterrows():
    img_path = os.path.join("Satriadata/preprocessing/", str(row[col_name]))
    true_plate = str(row[col_kunci])

    if not os.path.exists(img_path):
        print("⚠️ File tidak ditemukan:", img_path)
        continue

    # Prediksi
    results = model.predict(img_path, conf=0.25, verbose=False)  
    r = results[0]
    boxes = r.boxes
    xyxy = boxes.xyxy.cpu().numpy()
    cls = boxes.cls.cpu().numpy().astype(int)

    if len(cls) == 0:
        pred_plate = ""
    else:
        # urutkan kiri ke kanan
        sorted_idx = xyxy[:, 0].argsort()
        pred_plate = "".join([model.names[c] for c in cls[sorted_idx]])

    # hitung similarity dengan Levenshtein
    lev_dist = Levenshtein.distance(true_plate, pred_plate)
    max_len = max(len(true_plate), len(pred_plate)) if max(len(true_plate), len(pred_plate)) > 0 else 1
    perc_correct = (1 - lev_dist / max_len) * 100

    results_list.append({
        "File": row[col_name],
        "Kunci": true_plate,
        "Deteksi": pred_plate,
        "Persentase Benar": round(perc_correct, 2)
    })

# simpan hasil
results_df = pd.DataFrame(results_list)
print(results_df)
results_df.to_excel("18hasil_prediksi.xlsx", index=False)

Kolom nama file: Name
Kolom kunci    : kunci


c:\Users\fajar\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


               File      Kunci   Deteksi  Persentase Benar
0     DataTest1.png   AD7034OE  AD7034OE            100.00
1     DataTest2.png    A9388EX  A9388EAX             87.50
2     DataTest3.png      B16TB     B16TB            100.00
3     DataTest4.png   B1661TKZ  B1661TKZ            100.00
4     DataTest5.png  AD3772ABE  AD372ABE             88.89
..              ...        ...       ...               ...
95   DataTest96.png    B1285UL      7579             14.29
96   DataTest97.png   AB8644PK  AB8644PK            100.00
97   DataTest98.png   AG9718EG  AG9718EO             87.50
98   DataTest99.png    B1509UN   B1509UN            100.00
99  DataTest100.png    B1408RX   B1408RX            100.00

[100 rows x 4 columns]


In [7]:
# hitung rata-rata persentase benar dari seluruh data test
avg_score = results_df["Persentase Benar"].mean()

print(f"Rata-rata persentase benar: {avg_score:.2f}%")

Rata-rata persentase benar: 88.17%


In [8]:
# Jalankan evaluasi dengan dataset yang benar
metrics = model.val(data="charDetection-2\data.yaml", split='val')

print("Precision :", metrics.results_dict['metrics/precision(B)'])
print("Recall    :", metrics.results_dict['metrics/recall(B)'])
print("mAP@0.5   :", metrics.results_dict['metrics/mAP50(B)'])
print("mAP@0.5:0.95 :", metrics.results_dict['metrics/mAP50-95(B)'])


Ultralytics 8.3.184  Python-3.9.12 torch-2.2.2+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
val: Fast image access  (ping: 0.10.0 ms, read: 80.026.0 MB/s, size: 9.8 KB)


val: Scanning D:\Kuliah\Semester 8\Portfolio\Portfolio_ObjectDetection_BDC\charDetection-2\valid\labels.cache... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<?, ?it/s]

WARNING Box and segment counts should be equal, but got len(segments) = 188, len(boxes) = 234. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:08<00:00,  4.23s/it]


                   all         32        234      0.838      0.918      0.926      0.708
                     0          7          8      0.825          1      0.967      0.777
                     1         26         32      0.925      0.938      0.933      0.644
                     2         12         14      0.894          1      0.976       0.75
                     3          9         10      0.989          1      0.995      0.814
                     4          7          8      0.758      0.875      0.962      0.763
                     5          9         11      0.906       0.88      0.888       0.58
                     6          7          8      0.932      0.875       0.88      0.797
                     7          9         10      0.844        0.8      0.737      0.597
                     8         10         12      0.773      0.917      0.945      0.662
                     9          9         12      0.947      0.833      0.899      0.745
                     